## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_key

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Medicine Hat,CA,2022-04-14 01:00:26,50.0501,-110.6683,22.71,63,100,1.01,overcast clouds
1,1,Veraval,IN,2022-04-14 01:00:27,20.9000,70.3667,77.68,81,0,10.71,clear sky
2,2,Rumonge,BI,2022-04-14 01:00:28,-3.9736,29.4386,71.10,82,12,5.50,few clouds
3,3,Atuona,PF,2022-04-14 01:00:28,-9.8000,-139.0333,78.98,78,69,18.30,light rain
4,4,Pundaguitan,PH,2022-04-14 01:00:29,6.3711,126.1689,82.42,70,100,6.26,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Veraval,IN,2022-04-14 01:00:27,20.9000,70.3667,77.68,81,0,10.71,clear sky
3,3,Atuona,PF,2022-04-14 01:00:28,-9.8000,-139.0333,78.98,78,69,18.30,light rain
4,4,Pundaguitan,PH,2022-04-14 01:00:29,6.3711,126.1689,82.42,70,100,6.26,overcast clouds
5,5,Rikitea,PF,2022-04-14 01:00:29,-23.1203,-134.9692,77.85,71,12,18.34,light rain
7,7,Alofi,NU,2022-04-14 00:55:39,-19.0595,-169.9187,82.29,83,75,2.30,broken clouds
11,11,Lorengau,PG,2022-04-14 01:00:32,-2.0226,147.2712,85.77,69,99,9.10,overcast clouds
13,13,Padang,ID,2022-04-14 01:00:32,-0.9492,100.3543,78.85,76,52,1.68,broken clouds
15,15,Puerto Ayora,EC,2022-04-14 01:00:33,-0.7393,-90.3518,76.89,88,99,6.82,overcast clouds
16,16,Bambous Virieux,MU,2022-04-14 01:00:33,-20.3428,57.7575,79.21,83,69,19.48,light rain
19,19,Salvador,SV,2022-04-14 01:00:00,13.8333,-88.9167,75.06,67,78,3.47,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

City_ID        219
City           219
Country        219
Date           219
Lat            219
Lng            219
Max Temp       219
Humidity       219
Cloudiness     219
Wind Speed     219
Description    219
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Veraval,IN,77.68,clear sky,20.9000,70.3667,
3,Atuona,PF,78.98,light rain,-9.8000,-139.0333,
4,Pundaguitan,PH,82.42,overcast clouds,6.3711,126.1689,
5,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,
7,Alofi,NU,82.29,broken clouds,-19.0595,-169.9187,
11,Lorengau,PG,85.77,overcast clouds,-2.0226,147.2712,
13,Padang,ID,78.85,broken clouds,-0.9492,100.3543,
15,Puerto Ayora,EC,76.89,overcast clouds,-0.7393,-90.3518,
16,Bambous Virieux,MU,79.21,light rain,-20.3428,57.7575,
19,Salvador,SV,75.06,broken clouds,13.8333,-88.9167,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search\WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description/dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
output_gmap = "Vacation_Search\WeatherPy_vacation_map.png"
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))